# 6.2 Contact Problems

work in progress ...

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.solvers import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

rect = MoveTo(0,-0.02).Rectangle(0.04, 0.02).Face()
ball = MoveTo(0,0).Arc(0.005, 90).Rotate(90).Line(0.005).Close().Face()

rect.edges.Min(X).name="sym"
rect.edges.Max(X).name="free"
rect.edges.Min(Y).name="fix"
rect.edges.Max(Y).name="contact1"
ball.edges.Min(X).name="sym"
ball.edges.Max(Y).name="disp"
ball.edges.Max(X-Y).name="contact2"
ball.faces.name="ball"
rect.faces.name="rect"
rect = Glue ([rect, Vertex((0.001,0,0))])
rect.vertices.Max(Y-X).maxh=0.0002
rect.edges.Max(Y-0.1*X).maxh=0.00002
geo = Compound([rect,ball])
DrawGeo (geo)
mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=0.002))
mesh.Curve(4)
Draw(mesh);

In [ ]:
mesh.GetBoundaries(), mesh.GetMaterials()

In [ ]:
E = 2.1e11
nu = 0.2

rho = 7e3
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

I = Id(mesh.dim)
def Pow(a, b):
    return a**b  # exp (log(a)*b)

def C(u):
    # F = Id(2) + Grad(u)
    G = Grad(u)
    F = Id(3) + CoefficientFunction( (G[0,0], G[1,0], 0,   G[1,0], G[1,1], 0,    0, 0, u[0]/(x+1e-16) ) )
    return F.trans*F

def NeoHooke (C):
    return 0.5 * mu * (Trace(C-Id(C.dims[0])) + 2*mu/lam * Det(C)**(-lam/2/mu) - 1)

fes = VectorH1(mesh, order=4, dirichletx="sym", dirichlety="fix|disp")
u,v = fes.TnT()

a = BilinearForm(fes)
a += Variation((x*NeoHooke(C(u))).Compile()*dx)

contact = ContactBoundary(mesh.Boundaries("contact1"), mesh.Boundaries("contact2"))

X = CoefficientFunction((x,y))
cf = (X + u - (X.Other() + u.Other())) * contact.normal

# energy formulation
# contact.AddEnergy(IfPos(cf, 1e14*cf*cf, 0))
# or integrator
contact.AddIntegrator(IfPos(cf, 1e14*cf*(v-v.Other())*contact.normal, 0))

gfu = GridFunction(fes)

disp = -10e-6
gfu.Set((0, disp), definedon=mesh.Materials("ball"))

sceneu = Draw(gfu, deformation=gfu)

C_ = C(gfu)
sigma = NeoHooke(C_).Diff(C_)
scenesigma = Draw(sigma[1,1], mesh, "sigmazz", min=-2e9, max=2e9)


with TaskManager():
    contact.Update(gfu, a, 200, 1e-4)

    NewtonMinimization(a=a, u=gfu, printing=False, inverse="sparsecholesky")
    sceneu.Redraw()
    scenesigma.Redraw()

Draw(sigma[1,1], mesh, "sigmazz", min=-5e9, max=0, deformation=gfu);

## Dynamic Contact

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.solvers import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

In [ ]:
bowl = MoveTo(0,-0.01).Arc(0.2,70).Rotate(90).Line(0.01).Rotate(90) \
    .Arc(0.19,-140).Rotate(90).Line(0.01).Rotate(90).Arc(0.2,70).Face()
ball1 = Circle((0,0.1),0.01).Face()
ball2 = Circle((0.05,0.1),0.01).Face()
ball3 = Circle((-0.05,0.1),0.01).Face()
balls = Compound([ball1, ball2, ball3])
balls.edges.name = "balls"
geo = Compound([bowl, balls])
bowl.edges.name="contact"
bowl.edges.Min(Y+0.01*X).name="fix"
bowl.edges.Min(Y-0.01*X).name="fix"
DrawGeo(geo);

In [ ]:
mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=0.005))
mesh.Curve(4)
Draw(mesh);

In [ ]:
mesh.GetBoundaries()

In [ ]:
E, nu = 210e6, 0.2
rho = 7e3
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

I = Id(mesh.dim)

def C(u): 
    F = I+Grad(u)
    return F.trans*F
def NeoHooke (C):
    return 0.5 * mu * (Trace(C-I) + 2*mu/lam * Det(C)**(-lam/2/mu) - 1)

fes = VectorH1(mesh, order=3, dirichlet="fix")
u,v = fes.TnT()

force = CF((0,-9.81*rho))

uold = GridFunction(fes)
unew = GridFunction(fes)
vel = GridFunction(fes)
anew = GridFunction(fes)
aold = GridFunction(fes)

tau = 2e-4

bfmstar = BilinearForm(fes)
bfmstar += Variation( NeoHooke (C(u)).Compile(False)*dx )
bfmstar += Variation( -force*u*dx )
bfmstar += Variation( rho/2* 2/tau**2 * (u-uold-tau*vel-tau**2/4*aold)**2 * dx )

tend = 1

scene = Draw (unew, mesh, "disp", deformation=unew)

t = 0
unew.Set( (0,0) )
vel.Set( (0,0) )
contact = ContactBoundary(mesh.Boundaries("contact|balls"), mesh.Boundaries("contact|balls"))

X = CoefficientFunction((x,y))

if True:
    cf = (X + u-uold - (X.Other() + u.Other() - uold.Other())) * (-specialcf.normal(2).Other())
    # cf = (X + u-uold)*specialcf.normal(2) + \
    #    (X.Other() + u.Other() - uold.Other()) * specialcf.normal(2).Other()
    contact.AddEnergy(IfPos(cf, 1e9*cf*cf, 0), deformed=True)
else:
    cf = (X + u - (X.Other() + u.Other())) * contact.normal
    contact.AddEnergy(IfPos(cf, 1e9*cf*cf, 0), deformed=False)


In [ ]:
with TaskManager():
    while t < tend:
        print ("time", t)
        t += tau
        uold.vec.data = unew.vec
        aold.vec.data = anew.vec

        contact.Update(uold, bfmstar, 5, 0.01)
        NewtonMinimization (a=bfmstar, u=unew, printing=False, inverse="sparsecholesky")

        anew.vec.data = unew.vec-uold.vec-tau*vel.vec-tau**2/4*aold.vec
        anew.vec.data *= 4/tau**2
        vel.vec.data += 0.5*tau*aold.vec
        vel.vec.data += 0.5*tau*anew.vec

        scene.Redraw()

## A 3D example

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.solvers import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

In [ ]:
box = Box((0,0,0), (1,1,0.5))
rod = Cylinder((0.5, 0.5, -0.5),Z,0.2,1.5)
rod.faces.name="contact1"
block = box-rod
block.faces.Min(X).name="fix"
rod.faces.name="contact2"
rod.faces.Max(Z).name="force"
rod.faces.Min(Z).name="free"
geo = Compound([block, rod])
DrawGeo (geo);
mesh = Mesh(OCCGeometry(geo).GenerateMesh(maxh=0.1)).Curve(3)
Draw(mesh);

In [ ]:
mesh.GetBoundaries()

In [ ]:
fes = VectorH1(mesh, order=3, dirichlet="fix")
u = fes.TrialFunction()
gfu = GridFunction(fes)

a = BilinearForm(fes)
a += Variation(InnerProduct(Sym(Grad(u)), Sym(Grad(u)))*dx)
a += Variation(1e-8*u*u*dx)   # regularization
a += Variation(-0.01*u[0]*ds("force"))

In [ ]:
contact = ContactBoundary(mesh.Boundaries("contact1"), mesh.Boundaries("contact2"))
X = CoefficientFunction((x,y,z))
cf = (X + u - (X.Other() + u.Other())) * specialcf.normal(3)
contact.AddEnergy(IfPos(cf, 1e4*cf*cf, 0))
contact.Update(gfu, a, 20, 4)

In [ ]:
with TaskManager(pajetrace=10**9):
    NewtonMinimization(a=a, u=gfu, printing=True, inverse="sparsecholesky");

In [ ]:
Draw (gfu, mesh, deformation=True);
Draw (Sym(Grad(gfu))[0,0], mesh, draw_surf=False);